<a href="https://colab.research.google.com/github/parastooAflaki/Comment-Varification/blob/master/Commentverification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hazm
import numpy as np 
import pandas as pd 
from hazm import *
import string

In [0]:
from google.colab import files
uploadedTrainFile = files.upload()

Saving train_comments.csv to train_comments (1).csv


In [0]:
TrainDataFrame = pd.read_csv('train_comments (1).csv') #oon file i ke upload kardimo tooye ye dataFrame mirizim
print(TrainDataFrame.shape)

(180000, 5)


In [0]:
TrainDataFrame = TrainDataFrame.dropna() 
print(TrainDataFrame.shape)

(175079, 5)


In [0]:
def proccess_words(comment):
  wordsArray = [word for word in comment.split()] 
  return wordsArray

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
commentsBagOfWords = CountVectorizer(analyzer=proccess_words).fit_transform(TrainDataFrame['comment']) #in miad hamoon jadvale ke tooye voice goftamo misaaze be in jadvale estelahan migan bag of words
print(commentsBagOfWords.shape)

(175079, 174099)


In [0]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(commentsBagOfWords, TrainDataFrame['verification_status'], test_size = 0.20, random_state = 0)

In [0]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(xTrain, yTrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(xTrain)
print('Accuracy in train set: ', accuracy_score(yTrain,pred))

Accuracy in train set:  0.911354176334935


In [0]:
pred = classifier.predict(xTest)
print('Accuracy in train set: ', accuracy_score(yTest,pred))

Accuracy in train set:  0.8854238062599954


In [0]:
testFile = files.upload()

Saving test_nolabel_comments.csv to test_nolabel_comments.csv


In [0]:
testFrame = pd.read_csv('test_nolabel_comments.csv')

In [0]:
print(testFrame.shape)
testFrame['verification_status'] = 1 
concated = pd.concat([TrainDataFrame , testFrame])
concated = concated.fillna("") 

(18000, 5)
(18000, 5)


In [0]:
bagOfAllWords = CountVectorizer(analyzer=proccess_words).fit_transform(concated['comment']) 
testBOW = bagOfAllWords[concated.shape[0] - 18000: , :]
X_train, X_test, y_train, y_test = train_test_split(bagOfAllWords, concated['verification_status'], test_size = 0.2, random_state = 0) # halaye test o train joda kon baram
classifier = MultinomialNB()
classifier.fit(X_train, y_train) # train
predTest = classifier.predict(testBOW) 
output = testFrame.assign(verification_status = predTest)

In [0]:
print(output['verification_status'].sum()) 
output = output.drop(['title', 'comment', 'rate'], axis=1) 
print(output.shape)

6772
(18000, 2)


In [0]:
output.to_csv('df.csv')
files.download('df.csv')